# 생성 모델을 위한 딥러닝
* 머신 러닝 모델은 이미지, 음악, 글의 통계적 **잠재 공간**(latent space)을 학습할 수 있다.

## LSTM으로 텍스트 생성하기
### 생성 RNN의 간단한 역사
### 시퀀스 데이터를 어떻게 생성할까?
* 텍스트 데이터를 다룰 때 토큰은 보통 단어 또는 글자이다.
* 이전 토큰들이 주어졌을 때 다음 토큰의 확률을 모델잉할 수 있는 네트워크를 **언어 모델**(language model)이라고 부른다.
---
* 언어 모델을 훈련하고 나면 이 모델에서 샘플링을 할 수 있다.(새로운 시퀀스를 생성한다.)
* 초기 텍스트 문자열을 주입하고(**조건 데이터**(conditioning data) 새로운 글자나 단어를 생성한다.(한 번에 여러 개의 토큰을 생성할 수도 있다.)
* 생성된 출력은 다시 입력 데이터로 추가된다.
* **글자 수준의 신경망 언어 모델**(character-level neural language model)

### 샘플링 전략의 중요성
* 텍스트를 생성할 때 다음 굴자를 선택하는 방법이 아주 중요하다.
* 단순한 방법은 항상 가장 높은 확률을 가진 글자를 선택하는 **탐욕적 샘플링**(greedy sampling)이다.
* 다음 글자의 확률 분포에서 샘플링하는 과정에 무작위성을 주입하는 방법이다. **확률적 샘플링**(stochastic sampling)
* 모델의 소프트맥스 출력은 확률적 샘플링에 사용하기 좋다.
* 이 전략에는 샘플링 과정에서 무작위성의 양을 조절할 방법이 없다.
* 샘플링 과정에서 확률의 양을 조절하기 위해 **소프트맥스 온도**(softmax temperature)라는 파라미터를 사용한다.
* 샘플링에 사용되는 확률 분포의 엔트로피를 나타낸다.
* temperature 값이 주어지면 다음과 같이 가중치를 적용하여 (모델의 소프트맥스 출력인) 원본 확률 분포에서 새로운 확률 분포를 계산한다.
```
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)
```

### 글자 수준의 LSTM 텍스트 생성 모델 구현
#### 데이터 전처리

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)
text = open(path).read().lower()
print('text size:', len(text))

600901/600901 [==============================] - 1s 2us/step
text size: 600901


In [3]:
maxlen = 60
step = 3

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('sequence length:', len(sentences))

chars = sorted(list(set(text)))
print('unique char:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('vectorize...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

sequence length: 200281
unique char: 59
vectorize...


C:\Users\posky\AppData\Local\Temp\ipykernel_22124\1099389294.py:18: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
C:\Users\posky\AppData\Local\Temp\ipykernel_22124\1099389294.py:19: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool

#### 네트워크 구성

In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

c:\Users\posky\anaconda3\envs\tf20\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


#### 언어 모델 훈련과 샘플링
* 훈련된 모델과 시드(seed)로 쓰일 간단한 텍스트가 주어지면 다암과 같아 반복하여 새로운 텍스트를 생성할 수 있다.
    1. 지금까지 생성된 텍스트를 ㅈ입하여 모델에서 다음 글자에 대한 확률 분포를 뽑는다.
    2. 특정 온도로 이 확률 분포의 가중치를 조정한다.
    3. 가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링한다.
    4. 새로운 글자를 생성된 텍스트의 끝에 추가한다.

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)

    seed_text = text[start_index: start_index + maxlen]
    print('--- seed text: "' + seed_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1,2]:
        print('------ temperature:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
1565/1565 [==============================] - 11s 5ms/step - loss: 1.9571
--- seed text: "the slowly ascending ranks and classes, in which,
through fo"
------ temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the senders of the sent and the sense the senders of the sent the such the strength of the most the such the strength and the strength of the such the something the sense the strength of the strength and the there in the such the strance and contranted the sent and the such the ears of the strul and the sense that it is the such the sen and an the simital and some of the sense the such the sen o
------ temperature: 0.5
the slowly ascending ranks and classes, in which,
through for the sumpares, and the socr something of the contimant for the fourtions him the olitions reposed in the fact, it is the tan the san be not think the sent the threating of an anithing and sore and the himself. it is the free of should the place to the such the man every 

C:\Users\posky\AppData\Local\Temp\ipykernel_22124\2163503365.py:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


t
i acugier worrtl-godomoorfulativions, whesetherny-corducy  ;
 coust you a jesp.
withicty, tratthosps maim
teancr, liscen?w.-histor"s pufpo utterly, rr-regulu
ptyone,
-bupive credroin evseca) heact survior-piness
-mex hum-ceast an vihgsdau?mer, in grand,a.

anow,
who
gake,
"tadows," rattogicsy
oy to secrless" eptimatiyny,". to rely
meakgenlong nutio vonciue wou,pl2nation 
epoch 14
1565/1565 [==============================] - 7s 5ms/step - loss: 1.3486
--- seed text: "the slowly ascending ranks and classes, in which,
through fo"
------ temperature: 0.2
the slowly ascending ranks and classes, in which,
through foreundard and standly and desire to the self-contemplation of the self-conterpt the single the states of the spirit of the spirit to the self-contemplations of the self-contempt the stands of the feeling of the self-conditional standard of the superficial the self-contemplations of the sense of the self-contemplation of the spirit to the spirit of the free instance of the spirit 

* 낮은 온도는 아주 반복적이고 예상되는 텍스트를 만든다.
* 높은 온도에서 생성된 텍스트는 아주 흥미롭고 놀라우며 창의적이기도 하다.
* 항상 다양한 샘플링 전략으로 실헝해 보아야 한다.
* 학습된 구조와 무작위성 사이에 균형을 잘 맞추면 흥미로운 것을 만들 수 있다.

### 정리
* 이전의 토큰이 주어지면 다음 토큰(들)을 예측하는 모델을 훈련하여 시퀀스 데이터를 생성할 수 있다.
* 텍스트의 경우 이런 모델을 **언어 모델**이라고 부른다. 단어 또는 글자 단위 모두 가능하다.
* 다음 토큰을 샘플링할 때 모델이 만든 출력에 집중하는 것과 무작위성을 주입하는 것 사이에 균형을 맞추어야 한다.
* 이를 위해 소프트맥스 온도 개녕을 사용한다. 항상 다양한 온도를 실험해서 적절한 값을 찾는다.